In [1]:
import transformers
import numpy as np
import tensorflow as tf
import pandas as pd

# 1. Load BERT Model

Pretrained multi-lingual from HuggerFace

In [7]:
from transformers import BertTokenizer, TFBertModel

# Multi-lingual
bertMultTokeniser = BertTokenizer.from_pretrained('bert-base-uncased')
bertMultModel = TFBertModel.from_pretrained("bert-base-uncased")

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [11]:
bertMultModel.summary()

Model: "tf_bert_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
Total params: 109,482,240
Trainable params: 109,482,240
Non-trainable params: 0
_________________________________________________________________


## Test encoding text in various languages

In [69]:
sentences = [
    'Real Madrid is the best football team in the world',
    'Barcelona has a good team',
    'The scientists are satisfied with the experimental results',
    'The trip has been cancelled',
    'My friends and I loved travelling around the world'
]

tensors = [bertMultTokeniser(s, return_tensors='tf') for s in sentences]


tensors[0]

{'input_ids': <tf.Tensor: shape=(1, 12), dtype=int32, numpy=
array([[ 101, 2613, 6921, 2003, 1996, 2190, 2374, 2136, 1999, 1996, 2088,
         102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 12), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 12), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [70]:
wouts = [bertMultModel(t) for t in tensors]

In [71]:
[w[-1].numpy().shape for w in wouts] # pooling output

[(1, 768), (1, 768), (1, 768), (1, 768), (1, 768)]

Distance between each pair of sentences

In [80]:
from sklearn.metrics.pairwise import cosine_similarity

# Take weights of last pooling output
embeddings = [v[-1].numpy().flatten() for v in wouts]

zs = list(zip(sentences, embeddings))
from heapq import heappush, heappop

closest = []
for i,(s1,v1) in enumerate(zs):
    for j,(s2,v2) in enumerate(zs[i+1:]):
        c = cosine_similarity(
            np.atleast_2d(v1),
            np.atleast_2d(v2))
        heappush(closest, (-c, (s1,s2)))

In [81]:
print('Top 7 closest pairs')
for i in range(7):
    c, (s1,s2) = heappop(closest)
    print('==========================')
    print(f'Rank #{i}, {-c}')
    print(s1)
    print(s2)

Top 7 closes pairs
Rank #0, [[0.9878671]]
Real Madrid is the best football team in the world
Barcelona has a good team
Rank #1, [[0.9850803]]
The scientists are satisfied with the experimental results
The trip has been cancelled
Rank #2, [[0.9692514]]
Barcelona has a good team
My friends and I loved travelling around the world
Rank #3, [[0.96024734]]
Real Madrid is the best football team in the world
The trip has been cancelled
Rank #4, [[0.9584199]]
Real Madrid is the best football team in the world
My friends and I loved travelling around the world
Rank #5, [[0.9572299]]
Real Madrid is the best football team in the world
The scientists are satisfied with the experimental results
Rank #6, [[0.95030046]]
Barcelona has a good team
The scientists are satisfied with the experimental results
